In [1]:
from __future__ import division
import numpy as np
import glob
import numexpr

**10282**, **10651**, 10899, **11362**, **11808**, **11926**, **12040**, 12332, **9036**, **9622**, **10309**, **10668**, **11057**, **11499**, **11865**, **11937**, **12161**, 12379, **9255** , 10475, 10784, **11058**, **11637**, **11905** ,**1943**, 12268, 12482, ~~9507~~

**10670**, **11057**, **11058**, **11362**, **11499**, **11637**, **11808**, **11865**, **11905**, **11926**, **11937**, **11943**, **12040**, **12161**, 12268, **12332**, **12379**, ~~6514~~, ~~7017~~, **9036**, **9255**, ~~9507~~, **9622**, **10281**, **10282**, **10309**, 10475, **10651**, **10668**, **10784**, 10899, ~~12482~~, ~~12484~~ 

In [2]:
#Corsika 10670 - 2329   2265  144       600 - 1e5 GeV 
#Corsika 11058 - 28239  28239 395       1e5 - 1e11 GeV 
#Corsika 11057 - 74909                  600 - 1e5 GeV 
#Corsika 11362 - 34753  34684  1624     1e5 - 1e11 GeV 
#Corsika 11499 - 100000 99964  752      600 - 1e5 GeV 
#Corsika 11637 - 99683  99535  4536     1e5 - 1e11 GeV 
#Corsika 11808 - 100000 100000 671      600 - 1e5 GeV 
#Corsika 11865 - 100000 99860  739      600 - 1e5 GeV 
#Corsika 11905 - 100000 100000 753      600 - 1e5 GeV 
#Corsika 11926 - 100000 100000 708      600 - 1e5 GeV 
#Corsika 11937 - 100000 100000 45462    1e5 - 1e11 GeV 
#Corsika 11943 - 35929  33263  187      600 - 1e5 GeV 
#Corsika 12040 - 24800  23346  10672    1e5 - 1e11 GeV 
#Corsika 12161 - 99896  98026  736      600 - 1e5 GeV 
#Corsika 12268 - 99985  99985  1416     600 - 1e5 GeV 
#Corsika 12332 - 100000 98370  1527     600 - 1e5 GeV 
#Corsika 12379 - 34516  34516  937      600 - 1e5 GeV 
#Corsika 9036  - 99977  99977  627      1e5 - 1e11 GeV 	
#Corsika 9255  - 99986  99882  71395    1e5 - 1e11 GeV 	
#Corsika 9622  - 99999  99978  1140     600 - 1e5 GeV 
#Corsika 10281 - 97497  96383  1153     600 - 1e5 GeV 
#Corsika 10282 - 22866  22350  5694     3e4 - 1e11 GeV
#Corsika 10309 - 14368  14367  11869    600 - 1e11 GeV
#Corsika 10475 - 39337  38322  9558     3e4 - 1e11 GeV 
#Corsika 10651 - 39992  39841  9983     3e4 - 1e11 GeV 
#Corsika 10668 - 3955   3955   210      600 - 1e5 GeV 
#Corsika 10784 - 39990  38820  9647     3e4 - 1e11 GeV
#Corsika 10899 - 39812  39750  48226    1e5 - 1e11 GeV 	

In [2]:
electronvolt = 1.0e-9;
kiloelectronvolt = 1.e+3*electronvolt;
megaelectronvolt = 1.e+6*electronvolt; 
gigaelectronvolt = 1.e+9*electronvolt;
teraelectronvolt = 1.e+12*electronvolt;
petaelectronvolt = 1.e+15*electronvolt;
MeV = megaelectronvolt;
eV = electronvolt;
keV = kiloelectronvolt;
GeV = gigaelectronvolt;
TeV = teraelectronvolt;
PeV = petaelectronvolt;

class baseEnum(int):
    name = None
    values = {}
    def __repr__(self):
        return self.name

class metaEnum(type):
    def __new__(cls, classname, bases, classdict):
        newdict = {"values":{}}
        for k in classdict.keys():
            if not (k.startswith('_') or k == 'name' or k == 'values'):
                val = classdict[k]
                member = baseEnum(val)
                member.name = k
                newdict['values'][val] = member
                newdict[k] = member
              
        # Tell each member about the values in the enum
        for k in newdict['values'].keys():
            newdict['values'][k].values = newdict['values']
        # Return a new class with the "values" attribute filled
        return type.__new__(cls, classname, bases, newdict)

class enum(baseEnum, metaclass=metaEnum):
    """This class mimicks the interface of boost-python-wrapped enums.
      
Inherit from this class to construct enumerated types that can
be passed to the I3Datatype, e.g.:

        class DummyEnummy(tableio.enum):
                Foo = 0
               Bar = 1
                Baz = 2

       desc = tableio.I3TableRowDescription()
       desc.add_field('dummy', tableio.I3Datatype(DummyEnummy), '', '')
"""
class ParticleType(enum):
    PPlus       =   14
    He4Nucleus  =  402
    N14Nucleus  = 1407
    O16Nucleus  = 1608
    Al27Nucleus = 2713
    Fe56Nucleus = 5626
    NuE         =   66
    NuEBar      =   67
    NuMu        =   68
    NuMuBar     =   69
    NuTau       =  133
    NuTauBar    =  134

class PDGCode(enum):
    PPlus       =       2212
    He4Nucleus  = 1000020040
    N14Nucleus  = 1000070140
    O16Nucleus  = 1000080160
    Al27Nucleus = 1000130270
    Fe56Nucleus = 1000260560
    NuE         =         12
    NuEBar      =        -12
    NuMu        =         14
    NuMuBar     =        -14
    NuTau       =         16
    NuTauBar    =        -16
PDGCode.from_corsika = classmethod(lambda cls, i: getattr(cls, ParticleType.values[i].name))
ParticleType.from_pdg = classmethod(lambda cls, i: getattr(cls, PDGCode.values[i].name))
        
def build_lookup(mapping, var='ptype', default='ptype'):
    """
    Build an expression equivalent to a lookup table
    """
    if len(mapping) > 0:
        return 'where(%s==%s, %s, %s)' % (var, mapping[0][0], mapping[0][1], build_lookup(mapping[1:], var, default))
    else:
        return str(default)
class CompiledFlux(object):
    """
    An efficient pre-compiled form of a multi-component flux. For single-element evalutions
    this is ~2 times faster than switching on the primary type with an if statement; for 1e5
    samples it is 2000 times faster than operating on masked slices for each primary type.
    """
    pdg_to_corsika = numexpr.NumExpr(build_lookup([(int(PDGCode.from_corsika(v)), v) for v in ParticleType.values.keys()]))
    def __init__(self, expr):
        self.expr = numexpr.NumExpr(expr)
        # by default, assume PDG codes
        self._translator = CompiledFlux.pdg_to_corsika
       
    def to_PDG(self):
        """
        Convert to a form that takes PDG codes rather than CORSIKA codes.
        """
        new = copy.copy(self)
        new._translator = CompiledFlux.pdg_to_corsika
        return new
       
    def __call__(self, E, ptype):
        """
        :param E: particle energy in GeV
        :param ptype: particle type code
        :type ptype: int
        """
        if self._translator:
            ptype = self._translator(ptype)
            return self.expr(E, ptype)
       
    @staticmethod
    def build_lookup(mapping, var='ptype', default=0.):
        """
        Build an expression equivalent to a lookup table
        """
        # force mapping to be a list if it wasn't already
        mapping=list(mapping)
        if len(mapping) > 0:
            return 'where(%s==%s, %s, %s)' % (var, mapping[0][0], mapping[0][1], build_lookup(mapping[1:], var, default))
        else:
            return str(default)

class GaisserHillas(CompiledFlux):
    
    ptypes = [getattr(ParticleType, p) for p in ('PPlus', 'He4Nucleus', 'N14Nucleus', 'Al27Nucleus', 'Fe56Nucleus')]
    def get_expression(self, flux, gamma, rigidity):
        z = "where(ptype > 100, ptype%100, 1)"
        return "%(flux)s*E**(-%(gamma)s)*exp(-E/(%(rigidity)s*%(z)s))" % locals()
    def get_flux(self):
        return [[7860., 3550., 2200., 1430., 2120.]]
    def get_gamma(self):
        return [[2.66, 2.58, 2.63, 2.67, 2.63]]
    def get_rigidity(self):
        return [4*PeV]
    def __init__(self):
        flux = [self.build_lookup(zip(self.ptypes, f)) for f in self.get_flux()]
        gamma = [self.build_lookup(zip(self.ptypes, g)) for g in self.get_gamma()]
        rigidity = self.get_rigidity()
        CompiledFlux.__init__(self, "+".join([self.get_expression(f, g, r) for f, g, r in zip(flux, gamma, rigidity)]))

class GaisserH3a(GaisserHillas):  
    def get_flux(self):
        return super(GaisserH3a, self).get_flux() + [[20]*2 + [13.4]*3, [1.7]*2 + [1.14]*3]
    def get_gamma(self):
        return super(GaisserH3a, self).get_gamma() + [[2.4]*5, [2.4]*5]
    def get_rigidity(self):
        return super(GaisserH3a, self).get_rigidity() + [30*PeV, 2e3*PeV]

class GaisserH4a(GaisserH3a):
    def get_flux(self):
        return super(GaisserH4a, self).get_flux()[:-1] + [[200]]
    def get_gamma(self):
        return super(GaisserH4a, self).get_gamma()[:-1] + [[2.6]]
    def get_rigidity(self):
        return super(GaisserH4a, self).get_rigidity()[:-1] + [60e3*PeV]


In [3]:
data_type = 'corsika'
#n_files_in_folder = 2265+28239+74206+34684+99964+99535+100000+99860+100000+100000+100000+33263+23346+98026+99985+98370+34516
#print(n_files_in_folder)
name = "20904_corsika_test"
print(name)

N_PRIM_CHILDREN = 3 
STRINGS_TO_SAVE = 10
N_Y_BINS = 60
N_X_BINS = 500
N_CHANNELS = 3
outer_strings = set([1,2,3,4,5,6,7,13,14,21,22,30,31,40,41,50,51,59,60,67,68,72,73,74,75,76,77,78])

20904_corsika_test


In [4]:
def get_rates_corsika(cwm,nfiles):
    flux = GaisserH4a()
    pflux = flux (cwm["PrimaryEnergy"],cwm["PrimaryType"])
    energy_integral = (cwm['EnergyPrimaryMax']**(cwm["PrimarySpectralIndex"]+1)-cwm['EnergyPrimaryMin']**(cwm["PrimarySpectralIndex"]+1))/(cwm["PrimarySpectralIndex"]+1)
    energy_weight = cwm['PrimaryEnergy']**cwm["PrimarySpectralIndex"]
    secs_per_year = 31536000
    w = secs_per_year*pflux *energy_integral/energy_weight * cwm["AreaSum"] / (cwm['NEvents'] * nfiles)
    return w

In [5]:
id_dtype = np.dtype(
    [
        ("run_id", np.uint32),
        ("sub_run_id", np.uint32),
        ("event_id", np.uint32),
        ("sub_event_id", np.uint32),
    ]
)
preds_dtype = np.dtype(
    [     
        ('n1', np.float32),
        ('n2_1', np.float32),
        ('n2_2', np.float32),
        ('n2_3', np.float32),
        ('n3', np.float32)
    ]
)
st_info_dtype = np.dtype(
    [
        ('q', np.float32),
        ('num', np.uint32),
        ('dist', np.float32)
    ]
)
map_dtype = np.dtype(
    [
        ("id", id_dtype),
        ('raw', np.int32),
        ('st_raw', np.int32,(3)),
        ('pulses', np.int32),
        ('st_pulses', np.int32,(3)),
        ('cal', np.int32),
        ('st_cal', np.int32,(3)),
        ('hlc', np.int32),
        ('st_hlc', np.int32,(3)),
        ('slc', np.int32),
        ('st_slc', np.int32,(3))
    ]
)

particle_dtype = np.dtype(
    [
        ("tree_id", np.uint32,(2)),
        ("pdg", np.int32),
        ("energy", np.float32),
        ("position", np.float32,(3)),
        ("direction", np.float32,(2)),
        ("time", np.float32),
        ("length", np.float32)
    ]
)
veto_dtype = np.dtype(                                             
    [                                                                             
        ("SPE_rlogl", np.float32),                                                      
        ("Cascade_rlogl", np.float32),
        ("SPE_rlogl_noDC", np.float32),                                                   
        ("Cascade_rlogl_noDC", np.float32),                                              
        ("FirstHitZ", np.float32),
        ("VHESelfVetoVertexPosZ", np.float32),                                             
        ("LeastDistanceToPolygon_Veto", np.float32)
    ]
)

hese_dtype = np.dtype(                                             
    [                                                                             
 #       ("qtot", np.float32),
        ("vheselfveto", np.bool_),
        ("vheselfvetovertexpos", np.float32,(3)),
        ("vheselfvetovertextime", np.float32),
#        ("llhratio", np.float32)
    ]
)
CWEIGHT_KEY = "CorsikaWeightMap"
weight_dtype = np.dtype(
        [
            ("AreaSum" ,np.float32),
            ("Atmosphere",np.float32),
            ("CylinderLength",np.float32),
            ("CylinderRadius" ,np.float32),
            ("DiplopiaWeight",np.float32),
            ("EnergyPrimaryMax",np.float32),
            ("EnergyPrimaryMin",np.float32),
            ("FluxSum",np.float32),
            ("Multiplicity",np.float32),
            ("NEvents",np.float32),
            ("OldWeight",np.float32),
            ("OverSampling",np.float32),
            ("Polygonato",np.float32),
            ("PrimaryEnergy",np.float32),
            ("PrimarySpectralIndex",np.float32),
            ("PrimaryType",np.int64),
            ("ThetaMax",np.float32),
            ("ThetaMin" ,np.float32),
            ("TimeScale",np.float32),
            ("Weight",np.float32)
]                            
    )    
info_dtype = np.dtype(
    [
        ("id", id_dtype),
        ("image", np.float32, (N_X_BINS, N_Y_BINS, N_CHANNELS)),
        ("wf_times",np.float32,(N_Y_BINS, N_CHANNELS)),
        ("wf_pos",np.float32,(3, N_Y_BINS, N_CHANNELS)),
        ("wf_width",np.float32),
        ("qtot", np.float32),
        ("cog", np.float32,(3)),
        ("moi", np.float32),
        ("ti", np.float32,(4)),
        ("qst", st_info_dtype, N_CHANNELS),
        ("qst_all", st_info_dtype, STRINGS_TO_SAVE),
        ("map", map_dtype),
        ("primary", particle_dtype),
        ("prim_daughter", particle_dtype),
        ("trck_reco", particle_dtype),
        ("cscd_reco", particle_dtype),
        ("logan_veto", veto_dtype),
        ("hese_old", hese_dtype),
        ("hese", hese_dtype),
        ("llhcut",np.float32),
        ("weight", weight_dtype),
        ("weight_val",np.float32)
    ]
)

save_dtype = np.dtype(
    [
        ("id", id_dtype),
        ("preds", preds_dtype),
  #      ("wf_times",np.float32,(N_Y_BINS, N_CHANNELS)),
  #      ("wf_pos",np.float32,(3, N_Y_BINS, N_CHANNELS)),
  #      ("wf_width",np.float32),
        ("qtot", np.float32),
  #      ("cog", np.float32,(3)),
  #      ("moi", np.float32),
  #      ("ti", np.float32,(4)),
        ("qst", st_info_dtype, N_CHANNELS),
  #      ("qst_all", st_info_dtype, STRINGS_TO_SAVE),
  #      ("map", map_dtype),
        ("primary", particle_dtype),
        ("prim_daughter", particle_dtype),
   #     ("trck_reco", particle_dtype),
   #     ("cscd_reco", particle_dtype),
        ("logan_veto", veto_dtype),
   #     ("hese_old", hese_dtype),
        ("hese", hese_dtype),
        ("llhcut",np.float32),
        ("weight_val",np.float32)
    ]
)


In [6]:
import tensorflow as tf
import os

# Set which GPU to use.  This probably needs to be done before any other CUDA vars get defined.
# Use the command "nvidia-smi" to get association of a particular GPU with a particular number.
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "1,2,3,4"
from tensorflow.keras.models import load_model
model_1 = load_model('/home/dup193/work/double_pulse/AC922/vgg16_200k_Qst_2000_2/vgg16_200k_QSt2000_dataset_norm_2.h5')
model_2_1 = load_model('/home/dup193/work/double_pulse/AC922/vgg16_20k_Qst_2000_Corsika_20904/vgg16_20k_QSt2000_corsika_20904.h5')
model_2_2 = load_model('/home/dup193/work/double_pulse/AC922/vgg16_20k_Qst_2000_Corsika_20904_2/vgg16_20k_QSt2000_corsika_20904_2.h5')
model_2 = load_model('/home/dup193/work/double_pulse/AC922/vgg16_30k_Qst_2000_Corsika_3/vgg16_30k_QSt2000_corsika_3.h5')
model_3 = load_model('/home/dup193/work/double_pulse/AC922/vgg16_700k_Qst_2000_MuvsTau_4/vgg16_700k_QSt2000_dataset_norm_MuVsTau_3.h5')

mean_1 = 0.0012322452384978533 
std_1  = 0.009694634936749935
mean2_1 =  0.00025147481937892735
std2_1 = 0.005774625577032566
mean_2 = 0.0002565285249147564
std_2  = 0.005851939786225557
mean_3 = 0.00036459346301853657
std_3  = 0.007035365793853998

In [ ]:
size = 0
#f1 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_10670_100x/*.npz')
#f2 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11058_100x/*.npz')
#f3 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11057_100x/*.npz')
f4 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11362_100x/*.npz')
f5 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11499_100x/*.npz')
f6 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11637_100x/*.npz')
f7 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11808_100x/*.npz')
f8 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11865_100x/*.npz')
f9 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11905_100x/*.npz')
f10 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11926_100x/*.npz')
f11 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11937_100x/*.npz')
f12 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_11943_100x/*.npz')
f13 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12040_100x/*.npz')
f14 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12161_100x/*.npz')
f15 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12268_100x/*.npz')
f16 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12332_100x/*.npz')
f17 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12379_100x/*.npz')
f18 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_10668_100x/*.npz')

files_grabbed = f1+f2+f3+f4+f5+f6+f7+f8+f9+f10+f11+f12+f13+f14+f15+f16+f17+f18
print(len(files_grabbed))
for file_name in files_grabbed:
    x = np.load(file_name, mmap_mode="r")['arr_0']
    #print(file_name, len(x))
    size = size +len(x)
print(size)

In [7]:
size = 0
#f1 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_9036_100x/*.npz')
#f2 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_9255_100x/*.npz')
#f3 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_9622_100x/*.npz')
#f4 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12040_100x/*.npz')
#f5 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12161_100x/*.npz')
#f6 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12268_100x/*.npz')
#f7 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12332_100x/*.npz')
#f8 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12379_100x/*.npz')
#f9 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_20787_100x/*.npz')
#f10 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_20789_100x/*.npz')
#f11 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_20848_100x/*.npz')
f12 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_20904_100x_all/Images_Corsika_20904_00*.npz')
f13 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_20904_100x_all/Images_Corsika_20904_05*.npz')

#f13 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_20852_100x/*.npz')
#print(len(f1),len(f2),len(f3),len(f4),len(f5),len(f6),len(f7),len(f8),len(f9),len(f10),len(f11),len(f12),len(f13))

files_grabbed = f12+f13#+f2+f3+f4+f5+f6+f7+f8+f9+f10+f11+f12+f13
print(len(files_grabbed))
for file_name in files_grabbed:
    x = np.load(file_name, mmap_mode="r")['arr_0']
    #print(x[0].dtype)
    #print(file_name, len(x))
    size = size +len(x)
print(size)

50426
59311


In [8]:
weights = np.load("/home/dup193/work/double_pulse/Weighting_20904_test.npz", mmap_mode="r")['arr_0']
print(weights['weight'].shape)

(59311, 1)


In [9]:
print(sum(weights['weight']==0), size)

[0] 59311


In [11]:
import time
pos = 0
print(name)
start = time.time()
data = np.lib.format.open_memmap('/fastio2/dasha/double_pulse/data_'+name+'.npy', mode = 'w+', dtype =save_dtype, shape=(size,))
#n_files= len(glob.glob('/home/dup193/work/double_pulse/data/images_3str/Images_'+name+'*'))-1
#print(data.dtype)
#f1 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_9036_100x/*.npz')
#f2 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_9255_100x/*.npz')
#f3 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_9622_100x/*.npz')
#f4 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12040_100x/*.npz')
#f5 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12161_100x/*.npz')
#f6 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12268_100x/*.npz')
#f7 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12332_100x/*.npz')
#f8 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_12379_100x/*.npz')
#f9 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_20787_100x/*.npz')
#f10 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_20789_100x/*.npz')
#f11 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_20848_100x/*.npz')
#f12 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_20904_100x_all/*.npz')
f12 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_20904_100x_all/Images_Corsika_20904_00*.npz')
f13 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_20904_100x_all/Images_Corsika_20904_05*.npz')

#f13 = glob.glob('/home/dup193/work/double_pulse/data/images_3str/Corsika_20852_100x/*.npz')

#print(len(f1),len(f2),len(f3),len(f4),len(f5),len(f6),len(f7),len(f8),len(f9),len(f10),len(f11),len(f12),len(f13))
weights = np.load("/home/dup193/work/double_pulse/Weighting_20904_test.npz", mmap_mode="r")['arr_0']
files_grabbed = f12+f13#+f2+f3+f4+f5+f6+f7+f8+f9+f10+f11+f12+f13
print(len(files_grabbed),len(weights))
#print(np.unique(weights['set']))
count = 0
count_w = 0
for file_name in files_grabbed:
    x = np.load(file_name, mmap_mode="r")['arr_0']
    y = np.zeros(x.shape[0],dtype = save_dtype)
    
    im = (x['image']-mean_1)/std_1
    pred_n1 = model_1.predict([im[:,0,:,:,:1],im[:,0,:,:,1:2],im[:,0,:,:,2:3]],batch_size =1)
    im = (x['image']-mean2_1)/std2_1
    pred_n2_1 = model_2_1.predict([im[:,0,:,:,:1],im[:,0,:,:,1:2],im[:,0,:,:,2:3]],batch_size =1)
    pred_n2_2 = model_2_2.predict([im[:,0,:,:,:1],im[:,0,:,:,1:2],im[:,0,:,:,2:3]],batch_size =1)
    im = (x['image']-mean_2)/std_2
    pred_n2 = model_2.predict([im[:,0,:,:,:1],im[:,0,:,:,1:2],im[:,0,:,:,2:3]],batch_size =1)
    im = (x['image']-mean_3)/std_3
    pred_n3 = model_3.predict([im[:,0,:,:,:1],im[:,0,:,:,1:2],im[:,0,:,:,2:3]],batch_size =1)
    #print(file_name, len(x),len(y),pos)

    for n,e in enumerate(x):
        count +=1
        preds = np.zeros(1,dtype = preds_dtype)    
        preds[['n1','n2_1','n2_2','n2_3','n3']] = (pred_n1[n],pred_n2_1[n],pred_n2_2[n],pred_n2[n],pred_n3[n])
        
        if "PrimaryEnergy" in e['weight_dict'].dtype.fields:
            energy = e['weight_dict']['PrimaryEnergy']
        else:
            energy = e['primary']['energy']
            
        emask = weights['energy']==energy
        qmask = weights['qtot']==e['qtot']
        weight_vals = weights[emask & qmask]['weight']
        
        if len(weight_vals)==1:
            w = weight_vals[0]
        else:
            w = 0
         
        y[["id","preds","weight_val","qtot","qst","primary","prim_daughter","logan_veto","hese"]][n]\
        =(e['id'], preds, w ,e['qtot'],e['qst'],e['primary'],e['prim_daughter'],e['logan_veto'],e['hese'])
        
    data[pos:pos+len(x)] = y
    pos = pos + len(x)

end = time.time()
print(end - start)    


20904_corsika_test
50426 59311
11474.722355604172


In [ ]:
  ("moi", np.float32),
        ("ti", np.float32,(4)),
        ("map", map_dtype),
        ("qst", st_info_dtype, N_CHANNELS),
        ("qst_all", st_info_dtype, STRINGS_TO_SAVE),
        ("primary", particle_dtype),
        ("prim_daughter", particle_dtype),
        ("primary_child_energy", np.float32,(N_PRIM_CHILDREN)),
        ("primary_child_pdg", np.float32,(N_PRIM_CHILDREN)),
        ("logan_veto", veto_dtype),                                                  
        ("hese_old", hese_dtype),                                                  
        ("hese", hese_dtype),
        ("llhcut",np.float32),

In [14]:
data0= np.load('/fastio2/dasha/double_pulse/data_'+name+'.npy', mmap_mode='r')

In [15]:
print(data0['weight_val'].shape)

(33454,)


In [85]:
sum(data0['weight_val']==0)

1476

In [ ]:
import tensorflow as tf
import os

# Set which GPU to use.  This probably needs to be done before any other CUDA vars get defined.
# Use the command "nvidia-smi" to get association of a particular GPU with a particular number.
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "3,4"
from tensorflow.keras.models import load_model
model = load_model('/home/dup193/work/double_pulse/vgg16_600k_QSt2000_dataset_norm_muon_cut_3class_trial.h5')

mean = 0.0012322452384978533 
std = 0.009694634936749935
for file_name in glob.glob('/home/dup193/work/double_pulse/data/images_3str/Images_NuE_3_1_1_data.npz'):
    x = np.load(file_name, mmap_mode="r")['arr_0']
    for n,e in enumerate(x):    #mean = np.mean(im)
        im = (e['image']-mean)/std
        pred = model.predict([im[:,:,:,:1],im[:,:,:,1:2],im[:,:,:,2:3]])
       # print(pred) 

In [ ]:
  sumd = [-1,-1,-1]
        max_dom = [-1,-1,-1]
        cw_dom = [-1.0,-1.0,-1.0]
        sumd[0] =[np.sum(e['image'][0,:,i,0]) for i in range(60)]
        sumd[1] =[np.sum(e['image'][0,:,i,1]) for i in range(60)]
        sumd[2] =[np.sum(e['image'][0,:,i,2]) for i in range(60)]
        max_dom[0] = np.argmax(sumd[0])
        max_dom[1] = np.argmax(sumd[1])
        max_dom[2] = np.argmax(sumd[2])
        cw_dom[0] = np.average(range(1,61),weights = sumd[0])
        cw_dom[1] = np.average(range(1,61),weights = sumd[1])
        cw_dom[2] = np.average(range(1,61),weights = sumd[2])
        
        sumt = [-1,-1,-1]
        max_time = [-1,-1,-1]
        cw_time = [-1.0,-1.0,-1.0]
        sumt[0] =[np.sum(e['image'][0,i,:,0]) for i in range(500)]
        sumt[1] =[np.sum(e['image'][0,i,:,1]) for i in range(500)]
        sumt[2] =[np.sum(e['image'][0,i,:,2]) for i in range(500)]
        max_time[0] = np.argmax(sumt[0])
        max_time[1] = np.argmax(sumt[1])
        max_time[2] = np.argmax(sumt[2])
        cw_time[0] = np.average(range(1,501), weights = sumt[0])
        cw_time[1] = np.average(range(1,501), weights = sumt[1])
        cw_time[2] = np.average(range(1,501), weights = sumt[2])
        
        #print(cw_dom)
        #print(cw_time)
        #print(max_dom)
        #print(max_time)
      top_st = int(e['qst']['num'][0][0] in outer_strings)
        top3_st = sum([1 for i in e['qst']['num'][0] if i in outer_strings])
        top10_st = sum([1 for i in e['qst_all']['num'][0] if i in outer_strings])
        #print(top_st,top3_st,top10_st)
        
        im_sum = [-1,-1,-1]
        im_sum[0] = np.sum(e['image'][0,:,:,0])
        im_sum[1] = np.sum(e['image'][0,:,:,1])
        im_sum[2] = np.sum(e['image'][0,:,:,2])
        #print( e['qst']['num'][0])
        